In [3]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import os
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import xgboost

In [4]:
with open("ltable.csv") as file:
    ltable = pd.read_csv(file)
with open("rtable.csv") as file:
    rtable = pd.read_csv(file)
with open("train.csv") as file:
    train_data = pd.read_csv(file)

def preprocess_1(df, prepend):
#     df = df.drop(columns = ["price"])
    df = df.fillna('')
    df['extended_title'] = df['title'] + ' '+ df['category'] + ' '+ df['brand'] + ' ' + df["modelno"]
#     df = df.drop(columns = ["category", 'brand', 'modelno'])
    c = df.columns
    new_columns = [c[0]]
    for col in c[1:]:
        new_columns.append(prepend + col)
    df.columns = new_columns
    return df

ltable, rtable = preprocess_1(ltable, 'ltable_'), preprocess_1(rtable, 'rtable_')

In [159]:
test_data.head()


,ltable_id,rtable_id,id
0,1853,2139,0
1,1718,11835,8
2,1624,5013,9
3,614,5969,10
4,1512,19684,15


In [135]:
temp = train_data.merge(ltable, on=['ltable_id'], how='outer')
lhalf = temp.dropna(subset=['label'])
temp = lhalf.merge(rtable, on=['rtable_id'], how='outer')
full = temp.dropna(subset=['label'])
full = full.astype({"ltable_id": int, "rtable_id": int,"label": int, "id": int})

path_G = 'preprocessed.csv'
full.to_csv(path_G, index=False)


full.head(20)

,ltable_id,rtable_id,label,id,ltable_title,ltable_category,ltable_brand,ltable_modelno,ltable_price,ltable_extended_title,rtable_title,rtable_category,rtable_brand,rtable_modelno,rtable_price,rtable_extended_title
0,621,3167,0,1,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,88.88,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card electronics - general zotac zt-40604-10l,evga geforce gts450 superclocked 1 gb gddr5 pci-express 2.0 graphics card 01g-p3-1452-tr,graphics cards,evga,01g-p3-1452-tr,119.88,evga geforce gts450 superclocked 1 gb gddr5 pci-express 2.0 graphics card 01g-p3-1452-tr graphic...
1,1893,3167,0,3477,evga geforce 8400 gs 1gb ddr3 pci express 2.0 graphics card,electronics - general,evga,01g-p3-1302-lr,44.99,evga geforce 8400 gs 1gb ddr3 pci express 2.0 graphics card electronics - general evga 01g-p3-13...,evga geforce gts450 superclocked 1 gb gddr5 pci-express 2.0 graphics card 01g-p3-1452-tr,graphics cards,evga,01g-p3-1452-tr,119.88,evga geforce gts450 superclocked 1 gb gddr5 pci-express 2.0 graphics card 01g-p3-1452-tr graphic...
2,621,13148,0,194,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,88.88,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card electronics - general zotac zt-40604-10l,evga geforce 210 512 mb ddr3 pci-express 2.0 graphics card video card 512-p3-1310-lr,graphics cards,evga,512-p3-1310-lr,34.88,evga geforce 210 512 mb ddr3 pci-express 2.0 graphics card video card 512-p3-1310-lr graphics ca...
3,621,698,0,2260,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,88.88,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card electronics - general zotac zt-40604-10l,msi geforce 220gt 1024 mb ddr3 pci-express 2.0 graphics card md1gd3 lp,computers accessories,msi,n220gt-md1gd3 / lp,79.21,msi geforce 220gt 1024 mb ddr3 pci-express 2.0 graphics card md1gd3 lp computers accessories msi...
4,621,7701,0,3329,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,88.88,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card electronics - general zotac zt-40604-10l,pny geforce 9600gt 512 mb ddr3 pci-express graphics card vcg96512gxpb,,pny,vcg96512gxpb,,pny geforce 9600gt 512 mb ddr3 pci-express graphics card vcg96512gxpb pny vcg96512gxpb
5,934,7701,0,2086,evga geforce 210 512mb ddr3 pci express graphics card,electronics - general,evga,512p31310lr,32.88,evga geforce 210 512mb ddr3 pci express graphics card electronics - general evga 512p31310lr,pny geforce 9600gt 512 mb ddr3 pci-express graphics card vcg96512gxpb,,pny,vcg96512gxpb,,pny geforce 9600gt 512 mb ddr3 pci-express graphics card vcg96512gxpb pny vcg96512gxpb
6,621,11202,0,3557,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,88.88,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card electronics - general zotac zt-40604-10l,zotac zone geforce gt 220 1 gb 128-bit ddr2 625mhz 800mhz pci-express graphics card zt-20204-20l,computers accessories,zotac,zt-20204-20l,69.99,zotac zone geforce gt 220 1 gb 128-bit ddr2 625mhz 800mhz pci-express graphics card zt-20204-20l...
7,745,11202,0,1099,zotac geforce gt 220 synergy edition pci express 1gb ddr2 graphics card,electronics - general,zotac,zt-20203-10l,68.33,zotac geforce gt 220 synergy edition pci express 1gb ddr2 graphics card electronics - general zo...,zotac zone geforce gt 220 1 gb 128-bit ddr2 625mhz 800mhz pci-express graphics card zt-20204-20l,computers accessories,zotac,zt-20204-20l,69.99,zotac zone geforce gt 220 1 gb 128-bit ddr2 625mhz 800mhz pci-express graphics card zt-20204-20l...
8,621,13149,0,4331,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card,electronics - general,zotac,zt-40604-10l,88.88,zotac geforce gt430 1gb ddr3 pci-express 2.0 graphics card electronics - general zotac zt-40604-10l,evga geforce gt 430

In [146]:
def ratio(x):
    return fuzz.ratio(x[0], x[1])

def partial_ratio(x):
    return fuzz.partial_ratio(x[0], x[1])

def sort_ratio(x):
    return fuzz.token_sort_ratio(x[0], x[1])

diff = lambda x: (x[0] - x[1])**2

full['title_rat'] = full[['ltable_title', 'rtable_title']].apply(ratio, axis=1)
full['title_partial_rat'] = full[['ltable_title', 'rtable_title']].apply(partial_ratio, axis=1)
full['sort'] = full[['ltable_title', 'rtable_title']].apply(sort_ratio, axis=1)
full['ex_sort'] = full[['ltable_extended_title', 'rtable_extended_title']].apply(sort_ratio, axis=1)
full['ex_title_rat'] = full[['ltable_extended_title', 'rtable_extended_title']].apply(ratio, axis=1)
full['ex_title_partial_rat'] = full[['ltable_extended_title', 'rtable_extended_title']].apply(partial_ratio, axis=1)
full['modelno'] = full[['ltable_modelno', 'rtable_modelno']].apply(ratio, axis=1)
full['brand'] = full[['ltable_brand', 'rtable_brand']].apply(partial_ratio, axis=1)
# full['price'] = full[['ltable_price', 'rtable_price']].apply(diff, axis=1)
full.head()
final = full[['label', 'title_rat', 'title_partial_rat', 'sort','ex_title_rat','ex_title_partial_rat','ex_sort', 'modelno', 'brand']]
final.head()

,label,title_rat,title_partial_rat,sort,ex_title_rat,ex_title_partial_rat,ex_sort,modelno,brand
0,0,71,74,67,59,62,59,23,25
1,0,73,71,67,71,65,65,79,100
2,0,69,84,68,61,63,56,38,25
3,0,77,84,73,60,62,50,27,0
4,0,71,78,69,58,60,54,17,0


In [149]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import xgboost
%matplotlib inline

# method to encode data values to integers for training/predictions
def encode_data(data):
    return_data = pd.DataFrame()
    labelEncoder = preprocessing.LabelEncoder()
    for col in data.columns:
        return_data[col] = labelEncoder.fit_transform(data[col])
    return return_data

In [154]:
# import training data
training_data = final

# split training data between vector and labels
all_y = training_data.iloc[:,0]
all_x = training_data.iloc[:,1:]

# # use 20% of training data to test accuracy of training
train_x, test_x, train_y, test_y = train_test_split(all_x,all_y,test_size=.2)

In [156]:
all_y

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      1
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      1
23      0
24      0
25      0
26      0
27      0
28      1
29      0
       ..
7119    0
7120    0
7121    1
7122    1
7123    1
7124    1
7125    0
7126    1
7127    1
7128    1
7129    0
7130    1
7131    0
7132    0
7133    0
7134    1
7135    1
7136    0
7137    1
7138    0
7139    0
7140    0
7141    1
7142    0
7143    1
7144    0
7145    0
7146    0
7147    0
7148    1
Name: label, Length: 7149, dtype: int32

In [157]:
# train model
xgb =  XGBClassifier()
xgb.fit(train_x, train_y)
print ('Accuracy XGBoost: ' + str(accuracy_score(test_y, xgb.predict(test_x))))

Accuracy XGBoost: 0.9503496503496504


C:\Users\John\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
# import test data
test_noncoded = pd.read_csv("test.csv")
test_data = encode_data(test_noncoded)

# make predictions
predictions = list(xgb.predict(test_data))

# create submission file
classes = ['e' if p == 0 else 'p' for p in predictions]
test_noncoded['class'] = classes
submit = test_noncoded[['class','Id']]
submit.to_csv('submission.csv', index=False)
submit.head(10)

In [2]:
with open("test.csv") as file:
    test_data = pd.read_csv(file)